In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import transforms, datasets

In [ ]:
apply_transform = transforms.Compose([transforms.Resize(32), transforms.ToTensor()])
BatchSize = 250 # change according to system specs

trainset = datasets.MNIST(root='./MNIST', train=True, download=True, transform=apply_transform)
trainLoader = torch.utils.data.DataLoader(trainset, batch_size=BatchSize,
                                          shuffle=True, num_workers=4) # Creating dataloader

# Validation set with random rotations in the range [-90,90]
testset = datasets.MNIST(root='./MNIST', train=False, download=True, transform=apply_transform)
testLoader = torch.utils.data.DataLoader(testset, batch_size=BatchSize,
                                         shuffle=False, num_workers=4) # Creating dataloader

In [ ]:
# Size of train and test datasets
print('No. of samples in train set: '+str(len(trainLoader.dataset)))
print('No. of samples in test set: '+str(len(testLoader.dataset)))

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2)        
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 400)
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x,dim=1)

In [ ]:
use_gpu = torch.cuda.is_available()
net = LeNet()
print(net)
if use_gpu:
    print('GPU is avaialble!')
    net = net.cuda()

In [ ]:
#solution with gcd
Accur=[]
lr=0.0001
lear_rate=[]
train_loss=[[],[],[],[],[]]
train_acc=[[],[],[],[],[]]
for pq in range(0,3):
  lear_rate.append(lr)
  criterion = nn.CrossEntropyLoss()
  opt = torch.optim.SGD(net.parameters(), lr , momentum=0.9)
  num_epochs = 20
  for epoch in range(num_epochs):
      running_loss = 0.0 
      running_corr = 0
          
      for i,data in enumerate(trainLoader):
          inputs,labels = data
          if use_gpu:
              inputs, labels = inputs.cuda(),labels.cuda() 
          net.zero_grad()
          opt.zero_grad()
          outputs = net(inputs)
          preds = torch.argmax(outputs,dim=1)
          loss = criterion(outputs, labels) 
          loss.backward()
          running_loss += loss 
          running_corr += torch.sum(preds==labels)
          opt.step() 
      epoch_loss = running_loss.item()/(i+1)   
      epoch_acc = running_corr.item()/60000
      train_loss[pq].append(epoch_loss) 
      train_acc[pq].append(epoch_acc) 
      if((epoch+1)%5==0):
        print('Epoch {:.0f}/{:.0f} : Training loss: {:.4f} | Training Accuracy: {:.4f}'.format(epoch+1,num_epochs,epoch_loss,epoch_acc*100))  
      correct_pred = 0
      for data in testLoader:
          inputs,labels = data
          if use_gpu:
              inputs, labels = inputs.cuda(),labels.cuda()
          # Feedforward train data batch through model
          output = net(inputs) 
          # Predicted class is the one with maximum probability
          preds = torch.argmax(output,dim=1)
          correct_pred += torch.sum(preds==labels)

  test_accuracy = correct_pred.item()/10000.0
  Accur.append(test_accuracy*100)
  lr=lr+0.0001
  print(test_accuracy*100)

plt.plot(range(num_epochs),train_loss[0],'r',label="lr=0.0001") 
plt.plot(range(num_epochs),train_loss[1],'b',label="lr=0.0002") 
plt.plot(range(num_epochs),train_loss[2],'g',label="lr=0.0003") 
plt.xlabel('Epochs')
plt.ylabel('Training Error')
plt.legend()
plt.show()

plt.plot(range(num_epochs),train_acc[0],'r',label="0.0001") 
plt.plot(range(num_epochs),train_acc[1],'b',label="0.0002") 
plt.plot(range(num_epochs),train_acc[2],'g',label="lr=0.0003") 
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.legend()
plt.show()

In [ ]:
#solution with adam
Accur=[]
lr=0.0001
lear_rate=[]
train_loss=[[],[],[],[],[]]
train_acc=[[],[],[],[],[]]
for pq in range(0,3):
  lear_rate.append(lr)
  criterion = nn.CrossEntropyLoss()
  opt=   torch.optim.Adam(net.parameters(),lr = 0.002)
  num_epochs = 20
  for epoch in range(num_epochs):
      running_loss = 0.0 
      running_corr = 0
          
      for i,data in enumerate(trainLoader):
          inputs,labels = data
          if use_gpu:
              inputs, labels = inputs.cuda(),labels.cuda() 
          net.zero_grad()
          opt.zero_grad()
          outputs = net(inputs)
          preds = torch.argmax(outputs,dim=1)
          loss = criterion(outputs, labels) 
          loss.backward()
          running_loss += loss 
          running_corr += torch.sum(preds==labels)
          opt.step() 
      epoch_loss = running_loss.item()/(i+1)   
      epoch_acc = running_corr.item()/60000
      train_loss[pq].append(epoch_loss) 
      train_acc[pq].append(epoch_acc) 
      if((epoch+1)%5==0):
        print('Epoch {:.0f}/{:.0f} : Training loss: {:.4f} | Training Accuracy: {:.4f}'.format(epoch+1,num_epochs,epoch_loss,epoch_acc*100))  
      correct_pred = 0
      for data in testLoader:
          inputs,labels = data
          if use_gpu:
              inputs, labels = inputs.cuda(),labels.cuda()
          # Feedforward train data batch through model
          output = net(inputs) 
          preds = torch.argmax(output,dim=1)
          correct_pred += torch.sum(preds==labels)

  test_accuracy = correct_pred.item()/10000.0
  Accur.append(test_accuracy*100)
  lr=lr+0.0001
  print(test_accuracy*100)

plt.plot(range(num_epochs),train_loss[0],'r',label="lr=0.0001") 
plt.plot(range(num_epochs),train_loss[1],'b',label="lr=0.0002") 
plt.plot(range(num_epochs),train_loss[2],'g',label="lr=0.0003") 
plt.xlabel('Epochs')
plt.ylabel('Training Error')
plt.legend()
plt.show()
plt.plot(range(num_epochs),train_acc[0],'m',label="0.0001") 

plt.plot(range(num_epochs),train_acc[1],'b',label="0.0002") 
plt.plot(range(num_epochs),train_acc[2],'y',label="lr=0.0003") 
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.legend()
plt.show()